In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

!unzip '/content/drive/MyDrive/Colab Files/Car_YOLO.zip'

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: Car/images/Train/Gages Lake at North Mill East-7.jpg  
  inflating: Car/images/Train/Gages Lake at North Mill East-9.jpg  
  inflating: Car/images/Train/Gages Lake at North Mill West-0.jpg  
  inflating: Car/images/Train/Gages Lake at North Mill West-1.jpg  
  inflating: Car/images/Train/Gages Lake at North Mill West-2.jpg  
  inflating: Car/images/Train/Gages Lake at North Mill West-3.jpg  
  inflating: Car/images/Train/Gages Lake at North Mill West-4.jpg  
  inflating: Car/images/Train/Gages Lake at North Mill West-5.jpg  
  inflating: Car/images/Train/Gages Lake at North Mill West-6.jpg  
  inflating: Car/images/Train/Gages Lake at North Mill West-7.jpg  
  inflating: Car/images/Train/Gages Lake at North Mill West-9.jpg  
  inflating: Car/images/Train/Grand at McAree East-1.jpg  
  inflating: Car/images/Train/Grand at McAree East-2.jpg  
  inflating: Car/images/Train/Grand at McAree East-4.jpg  
  inflating: Car/images/Train/Grand at McAree Ea

In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 10.4 MB/s eta 0:00:00


In [3]:
import os
from ultralytics import YOLO

In [4]:
# Create YAML file
dataset_path = "/content/Car"
data_yaml = f"""
train: {dataset_path}/images/Train/
val: {dataset_path}/images/Val/
test: {dataset_path}/images/Test/
nc: 5
names: ['Truck','Sedan','SUV','Motorbike','Bus']
"""
with open("/content/Car/data.yaml", "w") as f:
    f.write(data_yaml)

In [5]:
train_folder = '/content/Car/images/Train/'
val_folder = '/content/Car/images/Val/'
test_folder = '/content/Car/images/Test/'

In [6]:
def create_txt_file(folder_path, txt_path):
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.jpg')])
    with open(txt_path, 'w') as file:
        for img_file in image_files:
            file.write(os.path.join(folder_path, img_file) + '\n')

In [7]:
create_txt_file(train_folder, '/content/Car/train.txt')
create_txt_file(val_folder, '/content/Car/val.txt')
create_txt_file(test_folder, '/content/Car/test.txt')

In [8]:
model = YOLO('yolov8n.yaml').load('yolov8n.pt')


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

In [9]:
results = model.train(data='/content/Car/data.yaml', epochs=100, imgsz=640)

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/Car/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimi

In [16]:
model = YOLO('/content/runs/detect/train/weights/best.pt')
model.export()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 168 layers, 3006623 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/content/runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 9, 8400) (5.9 MB)

TorchScript: starting export with torch 2.0.1+cu118...
TorchScript: export success ✅ 1.6s, saved as '/content/runs/detect/train/weights/best.torchscript' (11.9 MB)

Export complete (3.0s)
Results saved to /content/runs/detect/train/weights
Predict:         yolo predict task=detect model=/content/runs/detect/train/weights/best.torchscript imgsz=640  
Validate:        yolo val task=detect model=/content/runs/detect/train/weights/best.torchscript imgsz=640 data=/content/Car/data.yaml  
Visualize:       https://netron.app


'/content/runs/detect/train/weights/best.torchscript'

In [19]:
model = YOLO('/content/runs/detect/train/weights/best.pt')
model.export(format='engine')

WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3006623 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/content/runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 9, 8400) (5.9 MB)
requirements: Ultralytics requirement ['nvidia-tensorrt'] not found, attempting AutoUpdate...
Looking in indexes: https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.9/340.9 MB 286.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.1/333.1 MB 288.0 MB/s eta 0:00:

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



ONNX: export success ✅ 13.2s, saved as '/content/runs/detect/train/weights/best.onnx' (11.6 MB)

TensorRT: starting export with TensorRT 8.4.3.1...
TensorRT: input "images" with shape(1, 3, 640, 640) DataType.FLOAT
TensorRT: output "output0" with shape(1, 9, 8400) DataType.FLOAT
TensorRT: building FP32 engine as /content/runs/detect/train/weights/best.engine
TensorRT: export success ✅ 112.0s, saved as '/content/runs/detect/train/weights/best.engine' (14.8 MB)

Export complete (112.2s)
Results saved to /content/runs/detect/train/weights
Predict:         yolo predict task=detect model=/content/runs/detect/train/weights/best.engine imgsz=640  
Validate:        yolo val task=detect model=/content/runs/detect/train/weights/best.engine imgsz=640 data=/content/Car/data.yaml  
Visualize:       https://netron.app


'/content/runs/detect/train/weights/best.engine'

In [25]:
# Create YAML file
dataset_path = "/content/Car"
data_yaml = f"""
train: {dataset_path}/images/Train/
val: {dataset_path}/images/Test/
test: {dataset_path}/images/Val/
nc: 5
names: ['Truck','Sedan','SUV','Motorbike','Bus']
"""
with open("/content/Car/data.yaml", "w") as f:
    f.write(data_yaml)

model.val(data='/content/Car/data.yaml')  # Validate the model on the data you trained on

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
val: Scanning /content/Car/labels/Test... 292 images, 3 backgrounds, 0 corrupt: 100%|██████████| 295/295 [00:00<00:00, 1421.45it/s]
val: New cache created: /content/Car/labels/Test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  11%|█         | 2/19 [00:03<00:32,  1.91s/it]WARNING ⚠️ NMS time limit 1.300s exceeded
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  16%|█▌        | 3/19 [00:05<00:31,  1.95s/it]WARNING ⚠️ NMS time limit 1.300s exceeded
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:09<00:00,  2.03it/s]
                   all        295       2468      0.598       0.65      0.604      0.441
                 Truck        295        205      0.534      0.624      0.604      0.452
                 Sedan        295        929      0.3

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a1bcc690a00>
fitness: 0.45720005219344617
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.45201,     0.50729,     0.54531,     0.23103,     0.46863])
names: {0: 'Truck', 1: 'Sedan', 2: 'SUV', 3: 'Motorbike', 4: 'Bus'}
plot: True
results_dict: {'metrics/precision(B)': 0.5982461995705827, 'metrics/recall(B)': 0.6498425610458501, 'metrics/mAP50(B)': 0.6043307669954154, 'metrics/mAP50-95(B)': 0.4408521949932273, 'fitness': 0.45720005219344617}
save_dir: PosixPath('runs/detect/val3')
speed: {'preprocess': 0.5000526622190314, 'inference': 2.9783159999524136, 'loss': 0.0009164971820378707, 'postprocess': 9.969131825333934}